In [ ]:
#Import Necessary libraries
library('tidyverse')
library('dplyr')
library('forcats')
library('randomForest')
library('kableExtra')
source('functions.R')

In [ ]:
myTrain = function(datafr) {
    rf <- randomForest(
    return model
    }

In [ ]:
myPredict = function(datafr, model) {   ## second col must be lower bound 
    predict()
    }

In [ ]:
myScore = function(level, ...){
    
    
    }